## What is Terraform?
HashiCorp Terraform is an infrastructure as code tool that allows you to provision, model, and manage cloud and on-prem resources such as VMs, databases, containers, etc. using configuration files.

The benefits of using Terraform:
* Keep track of the creation and removal of infrastructure resources.
* Versioning and collaboration with version control (git).
* Reproducible infrastructre builds for testing and collaboration.

It does not:
* Does not manage or update code on infrastructure.
* Does not allow you to change immutable resources (e.g. upgrade VM or VM location).
* Does not manage resource not already defined in the terraform files.

## Installing Terraform
* Download the Terraform Windows client from https://developer.hashicorp.com/terraform/install or using WSL:
```default
wget -O- https://apt.releases.hashicorp.com/gpg | sudo gpg --dearmor -o /usr/share/keyrings/hashicorp-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/hashicorp-archive-keyring.gpg] https://apt.releases.hashicorp.com $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/hashicorp.list
sudo apt update && sudo apt install terraform
```

It's also recommended to install the [Terraform plugin](https://marketplace.visualstudio.com/items?itemName=HashiCorp.terraform) if using VSCode.

## Terraform Overview
Terraform uses **providers** to communicate with and manage cloud services. Many providers exist for popular cloud platforms (see the [Terraform Providers Registry](https://registry.terraform.io/browse/providers)).

```{mermaid}
flowchart LR
    A("Terraform") <-- Provider --> B("Cloud (GCP, AWS, or Azure)")
    subgraph "Local Machine"
    A
    end
```

Providers are essentially code libraries that read Terraform files and communicate with 3rd party services and platforms.

Helpful templates for each provider can be found at the [Terraform Registry](https://registry.terraform.io/).

## Creating Resources
Terraform config files are written in the HashiCorp Config Language (HCL) within `.tf` files. Terraform files are  structured as follows:

```terraform
# main.tf
# 1. First define a provider and authentication credentials (AWS for example).
provider "aws" {
    region = "us-west-2"
    access_key = "my-access-key"
    secret_key = "my-secret-key"
}

# 2. Then define resources (an EC2 instance for example )
#resource "<provider>_<resource_type>" "name" {
#    config options...
#    key = "value"
#}
resource "aws_instance" "first-server" {
    ami = ""
    instance = ""
    tags = {
        Name = "ec2-ubuntu-east-1"
    }
}
```

Note: 
- you can find the exact resource name in the provider documentation on the Terraform Registry site.
- the resource "name" is scoped to Terraform; only used for referencing this resource within Terraform.

## Running Terraform

Terraform includes several commands to set up provider plugins, test your config file, and run the configurations.

`terraform init` is going to parse the terraform file for listed providers and download the necessary provider plugins.

`terraform plan` will execute a "dry run" of the config file and show you the result of the actions defined in the config file without actually executing those actions.

`terraform apply` will first display the changes (resources created, stopped, or removed) to be made (another check of your configuration file). It will then prompt you to perform these actions.

Terraform files are **not** instructions carried out line by line, but they are a description of the final state of the infrastructure. Thus if the Terraform file is run multiple times and it will not keep creating new instances of resources. It will track changes and apply those changes, rather than rerun the entire config file.

## Deleting Resources
`terraform destroy` will remove *all* resources defined in our Terraform file.

We can also remove resources by deleting or commenting out resources within the Terraform file and running `terraform apply`.


## Resource Options

```terraform
provider "aws" {
    ...
}

resource "aws_vpc" "first-vpc" {
    cidr_block = "10.0.0.0/16"
    tags = {
        Name = "production"
    }
}

resource "aws_subnet" "subnet-1" {
    vpc_id = aws_vpc.first-vpc.id  # this line references the id of the resource created above "first-vpc"
    cidr_block = "10.0.1.0/24"

    tags = {
        Name = "prod-subnet"
    }
}
```

Resource can reference other resources no matter the order in which the resouce are defined.

## Other Terraform Files
* The `.terraform/plugins` folder is created when provider plugins are instantiated.
* The `terraform.tfstate` file keeps track of the state (additions, deletions) of the Terraform file.
* The `.terraform.lock.hcl` file includes hashes for the provider plugin to ensure you have the correct provider plugin.
* The `terraform.gitignore` file will block these sensitive terraform files from being uploaded to Github. 


## Example project: a webserver

1. Create VPC
2. Create Internet Gateway (to send traffic out to the internet)
3. Create Custom Route Table
4. Create a subnet
5. Associate subnet with Route Table
6. Create Security Group to allow port 22, 80, 443 (determine what kind of traffic is allowed)
7. Create a network interface with an ip in the subnet that was created in step 4
8. Assign an elastic IP to the network interface created in step 7 (a public IP address)
9. Create Ubuntu server and install/enable apache2 (and assign network interface)



## Extra Terraform Commands
- `terraform state list` : list all running resources
- `terraform state show <resource_name>` : show details of the running resource

- `terraform output` : print all of the outputs blocks without running `apply`. Use an `output` block in the Terraform file to print resource details to the console:
    ```terraform
    providers "aws" {
        ...
    }

    resource "resource" "name" {
        ...
    }

    output "public_ip" {
        value = aws_instance.first-server.public_ip
    }
    ```
- `terraform fmt` :  format your Terraform file.

## Targeting Resources
Individual resources can be targeted for specific instructions without creating, destroying, or modifying other defined resources in the Terraform file using the `-target` argument. For example:
```default
terraform destroy -target aws_instance.web-server-instance
```


## Terraform Variables
Terraform variables can be defined in a couple different ways. To define reusable variables in a Terraform file, use a `variables` block containing three optional arguments, `description`, `default`, and `type`.

```terraform
variable "subnet_prefix" {
    description = "cidr block for the subnet"
    # default
    type = string
}
```

Then to reference a variable, use `var.<variable_name>`.

```terraform
resource "aws_subnet" "subnet-1" {
    cidr_block = var.subnet_prefix  # reference defined variable
}
```

If a default value is not provided to the variable, Terraform will prompt the use to enter a value.

Variable values can also be passed using a command line argument.
```default
terraform apply -var "subnet_prefix=10.0.100.0/24"
```

Finally, variables can also be assigned with a separate `terraform.tfvars` file altogether.
```terraform
subnet_prefix = "10.0.200.0/24"
```

By default Terraform will look for a variables file called `terraform.tfvars`. To pass a custom named variables file, use the `-var-file` flag.
```default
terraform apply -var-file example.tfvars
```

## Terraform with Google Cloud
It is a best practice to create a service account specifically to use with Terraform.
- Create a service account *IAM & Admin > Service Accounts > Create Service Account*.
- Grant it access to *Cloud Storage > Storage Admin* and *Cloud SQL > BigQuery Admin*.
- Grab the key from *Manage Keys > Create a new key*.

```terraform
# main.tf

terraform {
    required_providers {
        google = {
            source = "hashicorp/google"
            version = "5.6.0"
        }
    }
}

provider "google" {
    project = "<Google project id>"
    region = "<nearest region>"
    credentials = file(var.credentials)
}

resources "google_storage_bucket" "first-bucket" {
    name = var.gcs_bucket_name
    location = "US"
    force_destroy = true
    
    # Add a lifecycle rule to shutdown long running uploads.
    lifecycle_rule {
        condition {
            age = 1
        }
        action {
            type = "AbortIncompleteMultipartUpload"
        }
    }
}

resources "google_bigquery_dataset" var.bq_dataset_name {
    dataset_id = "<dataset id>"
    friendly_name = "<readable name>"
    description = "<dataset description>"
    location = "US"
    default_table_expiration_ms = 3600000
}
```

```terraform
# variables.tf

variable "bq_dataset_name" {
    description = "BigQuery Dataset Name"
    default = "demo_dataset"
}


variable "gcs_bucket_name" {
    description = "Bucket Storage Name"
    default = "demo_bucket"
}

variable "credentials" {
    description = "my credentials"
    default = "./keys/creds.json"
    sensitive = true  # this marks this variable as sensitive and will not print it in logs.
}
```

## Using credentials in Terraform
The local solution for using credentials in Terraform is to use environment variables.

The cloud solution is to use cloud secret stores such as AWS KMS, GCP KMS, or Azure Secrets Vault.

It is not advised to use IAM user credentials with Terraform. IAM user credentials are usually static and long lived. This means that if the credentials are compromised they allow access for a long time and are difficult to revoke immediately.

The recommended alternative is OpenID Connect (OIDC). This comes with a few benefits:
1. Credentials are dynamically created for each run.
2. When Terraform Cloud authenticates with cloud providers using OIDC it will pass information about the project and run, to enforce IAM policies based on the context.
3. Credentials are short lived, they expire after the Terraform run completes.
4. You can immediately revoke access by removing the OIDC provider from cloud providers.
5. You don't need to manually export credentials from cloud providers and manage their rotation.

